In [ ]:
!apt update
!apt install zip unzip -y
!apt update
!apt install p7zip-full p7zip-rar -y

In [ ]:
%cd /workspace/content/DiffSinger
!pip install -r requirements.txt

In [ ]:
%cd /workspace/content/MakeDiffSinger/variance-temp-solution
!pip install -r requirements.txt

In [ ]:
!pip install pydub

# [Inference_Variance_0.1] **Preprocessing data (long segmentation)**

In [9]:
# Extract Data - this is because Variance needs transcription.csv having more columns
%cd /workspace/content
# This cell will create a folder name [raw_data_long] in the root folder and extract your data into it

data_type = "lab + wav (NNSVS format)" # @param ["lab + wav (NNSVS format)", "csv + wav (DiffSinger format)", "ds + wav (DiffSinger format)"]

# The path to your data zip file
data_zip_path = "/workspace/diffsinger_dataset.zip" #@param {type:"string"}

# nnsvs-db-converter settings (lab + wav ONLY)
# These values can exceed the amount that's in your data to maximize the segment length or to keep the data as is_
# This option is necessary for variance's pitch training
# Turn it on as you want to train Variance first. Turn it off later for Acoustics.
estimate_midi = True # for Variance
# estimate_midi = False #@param {type:"boolean"}

# Determine how long it will segment your data to based on silence phoneme placement (seconds)
segment_length = 35 #@param {type:"slider", min:5, max:35, step:1}

#Determine how many silence phoneme is allowed in the middle of each segment
max_silence_phoneme_amount = 50 #@param {type:"slider", min:0, max:50, step:1}

# leaving -S at 60 so max silence can be 60 seconds that exceeds the segment legnth cap idk why///
# making the segment length cap at 35 secs because any longer than that would make training goes really slow

# my ass dont remember why i made two... i think one is unnecessary extra but mehhh
all_shits = "/workspace/content/raw_data_long"
all_shits_not_wav_n_lab = "/workspace/content/raw_data_long/diffsinger_db"

import os
import csv
import json
import shutil
from pydub import AudioSegment

if os.path.exists("/workspace/content/raw_data_long"):
    shutil.rmtree("/workspace/content/raw_data_long")

if not os.path.exists(all_shits_not_wav_n_lab):
  os.makedirs(all_shits_not_wav_n_lab)

# using 'if not' bc i edited the wrong section which im also too lazy to fix it <3
if not data_type == "lab + wav (NNSVS format)":
    #changed to 7zip to support more compression types
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}
    for root, dirs, files in os.walk(all_shits):
        for filename in files:
            if filename.endswith(".lab"):
                file_path = os.path.join(root, filename)
                with open(file_path, "r") as file:
                    file_data = file.read()
                file_data = file_data.replace("SP", "pau")
                file_data = file_data.replace("br", "AP")
                with open(file_path, "w") as file:
                    file.write(file_data)

else:
    !7z x "$data_zip_path" -o{all_shits_not_wav_n_lab}


# for funny auto dict generator lmao
out = "/workspace/content/DiffSinger/dictionaries/custom_dict.txt"

phonemes = set()

def is_excluded(phoneme):
    return phoneme in ["pau", "AP", "SP", "sil"]

if data_type == "lab + wav (NNSVS format)":
    phoneme_folder_path = all_shits
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".lab"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as lab_file:
                    for line in lab_file:
                        line = line.strip()
                        if line:
                            phoneme = line.split()[2]
                            if not is_excluded(phoneme):
                                phonemes.add(phoneme)
elif data_type == "csv + wav (DiffSinger format)":
    phoneme_folder_path = all_shits_not_wav_n_lab
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".csv"):
                fpath = os.path.join(root, file)
                with open(fpath, "r", newline="") as csv_file:
                    csv_reader = csv.DictReader(csv_file)
                    for row in csv_reader:
                        if "ph_seq" in row:
                            ph_seq = row["ph_seq"].strip()
                            for phoneme in ph_seq.split():
                                if not is_excluded(phoneme):
                                    phonemes.add(phoneme)

else:
    phoneme_folder_path = all_shits_not_wav_n_lab
    for root, dirs, files in os.walk(phoneme_folder_path):
        for file in files:
            if file.endswith(".ds"):
                fpath = os.path.join(root, file)
                with open(fpath, "r") as json_file:
                    data = json.load(json_file)
                    for entry in data:
                        if "ph_seq" in entry:
                            ph_seq = entry["ph_seq"].strip()
                            phoneme_list = ph_seq.split()
                            for phoneme in phoneme_list:
                                if not is_excluded(phoneme):
                                    phonemes.add(phoneme)

with open(out, "w") as f:
    for phoneme in sorted(phonemes):
        f.write(phoneme + "	" + phoneme + "\n")

# for vowels and consonants.txt.... well adding luquid type for uta's script
dict_path = out
vowel_types = {"a", "i", "u", "e", "o", "N", "M", "NG"}
liquid_types = {"y", "w", "l", "r"} # r for english labels, it should be fine with jp too
vowel_data = []
consonant_data = []
liquid_data = []

with open(dict_path, "r") as f:
    for line in f:
        phoneme, _ = line.strip().split("\t")
        if phoneme[0] in vowel_types:
            vowel_data.append(phoneme)
        elif phoneme[0] in liquid_types:
            liquid_data.append(phoneme)
        else:
            consonant_data.append(phoneme)

vowel_data.sort()
liquid_data.sort()
consonant_data.sort()
directory = os.path.dirname(dict_path)

# make txt for language json file
vowel_txt_path = os.path.join(directory, "vowels.txt")
with open(vowel_txt_path, "w") as f:
    f.write(" ".join(vowel_data))
liquid_txt_path = os.path.join(directory, "liquids.txt")
with open(liquid_txt_path, "w") as f:
    f.write(" ".join(liquid_data))
consonant_txt_path = os.path.join(directory, "consonants.txt")
with open(consonant_txt_path, "w") as f:
    f.write(" ".join(consonant_data))


# here's a funny json append
with open(vowel_txt_path, "r") as f:
    vowel_data = f.read().split()
with open(liquid_txt_path, "r") as f:
    liquid_data = f.read().split()
with open(consonant_txt_path, "r") as f:
    consonant_data = f.read().split()
phones4json = {"vowels": vowel_data, "liquids": liquid_data}
with open("/workspace/content/nnsvs-db-converter/lang.sample.json", "w") as rawr:
    json.dump(phones4json, rawr, indent=4)


if data_type == "lab + wav (NNSVS format)":
    db_converter_script = "/workspace/content/nnsvs-db-converter/db_converter.py"
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        if os.path.isdir(raw_folder_path):
            if estimate_midi:
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -m -c -L "/workspace/content/nnsvs-db-converter/lang.sample.json" --folder {raw_folder_path}
            else:
                #!python {db_converter_script} -s 2 --folder {raw_folder_path} 2> /dev/null
                !python {db_converter_script} -s {max_silence_phoneme_amount} -S 60 -l {segment_length} -L "/workspace/content/nnsvs-db-converter/lang.sample.json" --folder {raw_folder_path}

else:
    pass


if data_type == "lab + wav (NNSVS format)":
    for raw_folder_name in os.listdir(all_shits_not_wav_n_lab):
        raw_folder_path = os.path.join(all_shits_not_wav_n_lab, raw_folder_name)
        !rm -rf {raw_folder_path}/*.wav {raw_folder_path}/*.lab
        !mv {raw_folder_path}/diffsinger_db/* {raw_folder_path} 2> /dev/null
        !rm -rf {raw_folder_path}/diffsinger_db
        #!cp {raw_folder_path}/wavs/*.wav {raw_folder_path}
else:
    pass

# make it replace the first SP to AP cus it seems like people always forgot about it
for root, _, files in os.walk(all_shits_not_wav_n_lab):
    for file in files:
        if file.endswith(".csv"):
            file_path = os.path.join(root, file)
            with open(file_path, "r", newline="") as input_file:
                csv_reader = csv.reader(input_file)
                data = [row for row in csv_reader]
                header = data[0]
                if "ph_seq" in header:
                    ph_seq_index = header.index("ph_seq")
                    if len(data) > 1 and len(data[1]) > ph_seq_index:
                        data[1][ph_seq_index] = data[1][ph_seq_index].replace("SP", "AP", 1)
            with open(file_path, "w", newline="") as output_file:
                csv_writer = csv.writer(output_file)
                csv_writer.writerows(data)

print("extraction complete!")
print("|")
print("|")
print("|")
print("I'm also nice enough to convert your data and also write your dict.txt lmao. You are welcome :)")

/workspace/content

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,48 CPUs AMD EPYC 7443P 24-Core Processor                (A00F11),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /workspace                    1 file, 143561117 bytes (137 MiB)

Extracting archive: /workspace/diffsinger_dataset.zip
--
Path = /workspace/diffsinger_dataset.zip
Type = zip
Physical Size = 143561117

     16% 11 - data/tiger_006.w                           31% 23 - data/tiger_012.w                           46% 29 - data/tiger_015.w                           61% 37 - data/tiger_019.w                           77% 47 - data/tiger_024.w                           92% 53 - data/tiger_027.w                          Everything is Ok

Files: 58
Size:       192500034
Compressed: 143561117
03/05/24 Tue 23:22:40 - INFO: Finding all labels.
03/05/24 Tue 23:22:40 - INFO: Found 28 labels.
03/05/24 Tue 23:22:40 - INFO: Making di

# [Inference_Variance_0.2] **Create sample (long segmentation)**

In [11]:
# transcription.csv now has columns to create .ds sample for inference
# create .ds sample for inference
!python /workspace/content/MakeDiffSinger/variance-temp-solution/convert_ds.py csv2ds /workspace/content/raw_data_long/diffsinger_db/data/transcriptions.csv /workspace/content/raw_data_long/diffsinger_db/data/wavs

0it [00:00, ?it/s]/workspace/content/MakeDiffSinger/variance-temp-solution/get_pitch.py:8: RuntimeWarning: divide by zero encountered in log2
  f0 = np.log2(f0)
72it [00:02, 24.42it/s]


In [ ]:
# Open /workspace/content/raw_data_long/diffsinger_db/data/wavs and choose .ds file you want
# for example: tiger_008_seg000.ds

# [Inference_Variance_1] **Edit Config for Variance Inference**

In [11]:
#Edit Config for VARIANCE
import re
import os
import yaml
import random #for the random test files lmaoz

%cd /workspace/content
# clear_output()
#@markdown <font size="-1.5"> The training type you want to do
config_type = "variance"
config_cap = config_type.upper()

spk_name = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
# print("spk_name", spk_name)
# remove '.ipynb_checkpoints' from spk_name
if ".ipynb_checkpoints" in spk_name:
    spk_name.remove(".ipynb_checkpoints")

# i used spk_name for something else cus i forgor now imma just copy and paste it
spk_names = [folder_name for folder_name in os.listdir(all_shits_not_wav_n_lab) if os.path.isdir(os.path.join(all_shits_not_wav_n_lab, folder_name))]
num_spk = len(spk_name)

raw_dir = []
for folder_name in spk_name:
    folder_path = os.path.join(all_shits_not_wav_n_lab, folder_name)
    raw_dir.append(folder_path)
if num_spk == 1:
    singer_type = "SINGLE-SPEAKER"
    diff_loss_type = "l2"
    f0_emb = "continuous"
    use_spk_id = False
    all_wav_files = []
    for root, dirs, files in os.walk("/workspace/content/raw_data_long/diffsinger_db"):
        for file in files:
            if file.endswith((".wav", ".ds")):
                full_path = os.path.join(root, file)
                all_wav_files.append(full_path)
    random.shuffle(all_wav_files)
    random_ass_wavs = all_wav_files[:3]
    random_ass_test_files = [os.path.splitext(os.path.basename(file))[0] for file in random_ass_wavs]

else:
    singer_type = "MULTI-SPEAKER"
    diff_loss_type = "l1"
    f0_emb = "discrete"
    use_spk_id = True
    folder_to_id = {folder_name: i for i, folder_name in enumerate(spk_name)}
    random_ass_test_files = []
    for folder_path in raw_dir:
        if data_type == "ds + wav (DiffSinger format)":
            audio_files = [f[:-4] for f in os.listdir(folder_path) if f.endswith(".ds")]
        else:
            audio_files = [f[:-4] for f in os.listdir(folder_path + "/wavs") if f.endswith(".wav")]
        folder_name = os.path.basename(folder_path)
        folder_id = folder_to_id.get(folder_name, -1)
        prefixed_audio_files = [f"{folder_id}:{audio_file}" for audio_file in audio_files]
        random_ass_test_files.extend(prefixed_audio_files[:3])
spk_id = []
for i, spk_name in enumerate(spk_name):
    spk_id_format = f"{i}:{spk_name}"
    spk_id.append(spk_id_format)

#@markdown <font size="-1.5"> Shallow Diffusion training for acoustic
use_shallow_diffusion = "true | gt_val" # @param ["false", "true | aux_val", "true | gt_val"]
if use_shallow_diffusion == "false":
    shallow = False
    gt_shallow = False
elif use_shallow_diffusion == "true | aux_val":
    shallow = True
    gt_shallow = False
else:
    shallow = True
    gt_shallow = True

#@markdown <font size="-1.5"> Half precision, or mixed precision can result in improved performance, achieving speedups on training (from [doc](https://lightning.ai/docs/pytorch/stable/common/trainer.html#precision))
# the reason why i dont add 64 is because colab is already dreadfully slow at 32 so yes im leaving it out
precision = "16-mixed" # @param ["32-true", "bf16-mixed", "16-mixed", "bf16", "16"]

#@markdown <font size="-1.5"> batch size setting, too low can cause bottleneck, too high can cause oom
batch_size = 12 # @param {type:"slider", min:1, max:100, step:1}


#@markdown <font size="-1.5"> Your model save path
save_dir = "/workspace/content/DiffSinger/MyModel_Variance"

binary_save_dir = save_dir + "/binary"

conf_dir = save_dir

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Option to use base model for finetuning

enable_finetuning = False # @param {type:"boolean"}


#@markdown <font size="-1.5"> Path to custom base model, leave blank to use [default](https://github.com/haru0l/diffsinger_models) models
#wtf haru i just looked at your readme"""""

base_model_path = "" # @param {type:"string"}

if enable_finetuning:
    pretrain = True
    if base_model_path:
        pretrain_ckpt = base_model_path
    else:
        pretrain_ckpt = f"/workspace/content/pretrain_models/{config_type}_pretrain.ckpt"
    finetune_strict_shapes = False
    finetune_ckpt_path = pretrain_ckpt
else:
    pretrain = False
    finetune_strict_shapes = True #default value
    finetune_ckpt_path = None #default value

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Model embeds check; Tension, Energy, Breathiness, Voicing | for both acoustic and variance
enable_embeds = True #@param {type: "boolean"}
if enable_embeds:
  tension_training = True
  energy_training = True
  breathiness_training = True
  voicing_training = True
else:
  tension_training = False
  energy_training = False
  breathiness_training = False
  voicing_training = False

# Model training check | for variance only
# @param ["False", "True | Standard", "True | Standard + Glide", "True | MelodyEncoder", "True | MelodyEncoder + Glide"]

pitch_training = "True | Standard" # for variance only
if pitch_training == "False":
    pitch_training = False
    use_melody_encoder = False
    use_glide_embed = False
elif pitch_training == "True | Standard":
    pitch_training = True
    use_melody_encoder = False
    use_glide_embed = False
elif pitch_training == "True | Standard + Glide":
    pitch_training = True
    use_melody_encoder = False
    use_glide_embed = True
elif pitch_training == "True | MelodyEncoder":
    pitch_training = True
    use_melody_encoder = True
    use_glide_embed = False
else:
    pitch_training = True
    use_melody_encoder = True
    use_glide_embed = True

duration_training = True #@param {type: "boolean"}

#@markdown ....................................................................................................................................................................................................................................................................................................................................................................................................................................

#@markdown <font size="-1.5"> Pitch extractor algorithm

f0_ext = "rmvpe" # @param ["parselmouth", "rmvpe", "harvest"]
if f0_ext == "rmvpe":
    pe_ckpt_pth = "checkpoints/rmvpe/model.pt"
else:
    pe_ckpt_pth = None

#@markdown <font size="-1.5"> Select this if you want to use data augmentation (default pitch shift and time stretch values)
data_aug = False # Variance #@param {type:"boolean"}

with open("/workspace/content/DiffSinger/configs/base.yaml", "r") as config:
    mother = yaml.safe_load(config)
mother["pl_trainer_precision"] = precision
with open("/workspace/content/DiffSinger/configs/base.yaml", "w") as config:
    yaml.dump(mother, config)

if  data_type == "ds + wav (DiffSinger format)":
    prefer_ds = True
else:
    prefer_ds = False

if config_type == "acoustic":
    with open("/workspace/content/DiffSinger/configs/acoustic.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    #bitch_ass_config["spk_ids"] = spk_id
    bitch_ass_config["diff_loss_type"] = diff_loss_type
    bitch_ass_config["f0_embed_type"] = f0_emb
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["augmentation_args"]["random_pitch_shifting"]["enabled"] = data_aug
    bitch_ass_config["augmentation_args"]["random_time_stretching"]["enabled"] = data_aug
    bitch_ass_config["use_key_shift_embed"] = data_aug
    bitch_ass_config["use_speed_embed"] = data_aug
    bitch_ass_config["max_batch_size"] = batch_size
    bitch_ass_config["pe"] = f0_ext
    bitch_ass_config["use_energy_embed"] = energy_training
    bitch_ass_config["use_breathiness_embed"] = breathiness_training
    bitch_ass_config["use_tension_embed"] = tension_training
    bitch_ass_config["use_voicing_embed"] = voicing_training

    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth
    #shallow diff stuff
    bitch_ass_config["use_shallow_diffusion"] = shallow
    bitch_ass_config["shallow_diffusion_args"]["val_gt_start"] = gt_shallow
    #pretrain stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes

    with open("/workspace/content/DiffSinger/configs/acoustic.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)
else:
    with open("/workspace/content/DiffSinger/configs/variance.yaml", "r") as config:
        bitch_ass_config = yaml.safe_load(config)
    bitch_ass_config["speakers"] = spk_names
    bitch_ass_config["test_prefixes"] = random_ass_test_files
    bitch_ass_config["raw_data_dir"] = raw_dir
    bitch_ass_config["num_spk"] = num_spk
    bitch_ass_config["use_spk_id"] = use_spk_id
    bitch_ass_config["diff_loss_type"] = diff_loss_type
    bitch_ass_config["binary_data_dir"] = binary_save_dir
    bitch_ass_config["dictionary"] = "dictionaries/custom_dict.txt"
    bitch_ass_config["max_batch_size"] = batch_size
    bitch_ass_config["pe"] = f0_ext # i think variance uses it for pitch ref as ground-truth for pitch training soooo
    bitch_ass_config["pe_ckpt"] = pe_ckpt_pth #same goes to this one

    bitch_ass_config["predict_energy"] = energy_training
    bitch_ass_config["predict_breathiness"] = breathiness_training
    bitch_ass_config["predict_tension"] = tension_training
    bitch_ass_config["predict_pitch"] = pitch_training
    bitch_ass_config["predict_voicing"] = voicing_training

    bitch_ass_config["use_melody_encoder"] = use_melody_encoder
    bitch_ass_config["use_glide_embed"] = use_glide_embed
    bitch_ass_config["predict_dur"] = duration_training
    bitch_ass_config["binarization_args"]["prefer_ds"] = prefer_ds
    #pretrain stuff
    bitch_ass_config["finetune_enabled"] = enable_finetuning
    bitch_ass_config["finetune_ckpt_path"] = finetune_ckpt_path
    bitch_ass_config["finetune_strict_shapes"] = finetune_strict_shapes

    with open("/workspace/content/DiffSinger/configs/variance.yaml", "w") as config:
        yaml.dump(bitch_ass_config, config)

os.makedirs(save_dir, exist_ok=True)


with open("/workspace/content/DiffSinger/utils/hparams.py", "r") as f:
    hparams_py_read = f.read()
hparams_py_read = re.sub(r"args_work_dir\s*=\s*.*", f"args_work_dir = '{save_dir}'", hparams_py_read)
with open("/workspace/content/DiffSinger/utils/hparams.py", "w") as f:
    f.write(hparams_py_read)

with open("/workspace/content/DiffSinger/utils/training_utils.py", "r") as f:
    training_utils_stuff = f.read()
training_utils_stuff = re.sub("relative_path\s*=\s*.*", "relative_path = filepath.relative_to(Path('/workspace/content').resolve())", training_utils_stuff)
with open("/workspace/content/DiffSinger/utils/training_utils.py", "w") as f:
    f.write(training_utils_stuff)

print("config updated! see below for config's information")
print("|")
print("|")
print("|")
print(f"+++---{config_cap} {singer_type} TRAINING---+++")
print("|")
print("|")
print("|")
print("+++---user's settings---+++")
print("\n")
print(f"speaker name: {spk_names}")
print("\n")
print(f"data augmentation: {data_aug}")
print("\n")
print(f"pitch extractor: {f0_ext}")
print("\n")
print(f"binary data save directory: {binary_save_dir}")
print("\n")
print(f"your model will be saved to: {save_dir}")
print("\n")
print("==========================================================================================")
print("\n")
print("+++---other auto-defined settings---+++")
print("\n")
print(f"test files (auto selected): {random_ass_test_files}")
print("\n")
print("dictionary (auto generated): custom_dict.txt")
print("\n")
print(f"max_batch_size: {batch_size}")
print("\n")
print("==========================================================================================")
print("\n")
print("if you don't like or disagree with any of these options,")
print(f"you can go and edit the config at [/workspace/content/DiffSinger/configs/{config_type}.yaml]")


/workspace/content
config updated! see below for config's information
|
|
|
+++---VARIANCE SINGLE-SPEAKER TRAINING---+++
|
|
|
+++---user's settings---+++


speaker name: ['data']


data augmentation: False


pitch extractor: rmvpe


binary data save directory: /workspace/content/DiffSinger/MyModel_Variance/binary


your model will be saved to: /workspace/content/DiffSinger/MyModel_Variance




+++---other auto-defined settings---+++


test files (auto selected): ['tiger_017_seg001', 'tiger_028_seg004', 'tiger_001_seg000']


dictionary (auto generated): custom_dict.txt


max_batch_size: 12




if you don't like or disagree with any of these options,
you can go and edit the config at [/workspace/content/DiffSinger/configs/variance.yaml]


# [Inference_Variance_2] **Variance Inference**

In [12]:

!python /workspace/content/DiffSinger/scripts/infer.py variance /workspace/tiger_008_seg000.ds --exp /workspace/content/DiffSinger/MyModel_Variance --out /workspace --title inference_variance_result_long

| found ckpt by name: /workspace/content/DiffSinger/MyModel_Variance
| Hparams chains:  []
| Hparams: 
K_step: 1000, accumulate_grad_batches: 1, audio_sample_rate: 44100, base_config: [], binarization_args: {'num_workers': 0, 'prefer_ds': False, 'shuffle': True}, 
binarizer_cls: preprocessing.variance_binarizer.VarianceBinarizer, binary_data_dir: /workspace/content/DiffSinger/MyModel_Variance/binary, breathiness_db_max: -20.0, breathiness_db_min: -96.0, breathiness_smooth_width: 0.12, 
clip_grad_norm: 1, dataloader_prefetch_factor: 2, dataset_size_key: lengths, dictionary: dictionaries/custom_dict.txt, diff_accelerator: ddim, 
diff_decoder_type: wavenet, diff_loss_type: l2, diff_speedup: 10, dropout: 0.1, ds_workers: 4, 
dur_prediction_args: {'arch': 'fs2', 'dropout': 0.1, 'hidden_size': 512, 'kernel_size': 3, 'lambda_pdur_loss': 0.3, 'lambda_sdur_loss': 3.0, 'lambda_wdur_loss': 1.0, 'log_offset': 1.0, 'loss_type': 'mse', 'num_layers': 5}, enc_ffn_kernel_size: 9, enc_layers: 4, energy_